# Estudio de las estaciones de toma de datos

> [Configuracion del notebook](#config)
>
> [Estaciones de españa](#espanha)
>    * [Datos de las estaciones de ```saqgetr```](#estacionesSaqgetr)
>    * [Datos geograficos de las estaciones](#estacionGeo)
>
> [Estudio de los Datos de las Estaciones](#stdio)

## Configuracion del notebook <a id="config"></a>

Cargamos todos los paquetes necesarios para ejecutar el notebook

In [6]:
suppressMessages(library(saqgetr))
suppressMessages(library(tidyverse))
suppressMessages(library(lubridate))

Inicializamos ciertas variables con los parametros de estudio

In [7]:
# contaminantes a estudiar
pollutants <- c("no", "no2", "o3", "pm10")

# fechas de inicio y final de toma de datos
start_dt <- ymd_hms("2010-01-01 00:00:00")
end_dt <- ymd_hms("2020-12-31 00:00:00")

# fecha de inicio de confinamiento
lckdwn_strt <- ymd_hms("2020-03-14 00:00:00")

## Estaciones de España <a id="espanha"></a>

### Datos geograficos de las estaciones <a id="estacionGeo"></a>

Seleccionamos aquellas estaciones que se encuentren en un núcleo con una
población mayor o igual a cien mil habitantes `poblacion >= 100 000 hab`.

En la hoja 8 `ciudades-100000-A-JA` de la base de datos de poblaciones (`estaciones-CA.xlsx`) aparecen todas las estaciones de trafico de las ciudades de mas de 100000 habitantes con sus nombres.

In [8]:
# Loading
library("readxl")

file <- "../data/xlsx/estaciones-CA-JA.xlsx"
sheets <- c("todas", "traffic", "traffic-urban", "traffic-urban-2020",
            "traffic-suburban", "traffic-suburban-2020",
            "ciudades-100000", "ciudades-100000-A")
# xlsx files
sites.100mil <- read_excel(file, sheet=sheets[8])

In [9]:
head(sites.100mil)

Municipio,Población,Estación tráfico,Código estación,Nº estaciones Ecolog,Nº estaciones tráfico,Observaciones
<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>
A Coruña,245711,CORLAB 1,es1138a,4,1,"Es Riazor o Santa Margarida, revisar. Datos sólo hasta 2018. REVISAR"
Alcalá de Henares,195649,Alcalá de Henares,es1563a,0,1,NA
Alcobendas,117040,Alcobendas,es1564a,0,1,NA
Alicante,334887,Florida-Babel,es1915a,3,1,Urban background. Not traffic
Alicante,334888,ALACANT-EL PLÁ,es1635a,3,1,Esta es de tráfico
Almería,198533,MEDITERRÁNEO,es1393a,0,1,NA


### Datos de las estaciones de `saqgetr` <a id="estacionesSaqgetr"></a>

Importamos la informacion de las estaciones de calidad de aire de españa obtenidas de la base de datos y filtramos segun los criterios de estudio.

| Variable | Valores |
|-------|-----------|
|Contaminantes| $NO$, $NO_2$, $O_3$, $PM_{10}$|
|Fecha Inicio| 01 Enero 2015 |
|Fecha Final | 31 Diciembre 2020 |
| Site Type | traffic |
|site area | urban |

In [10]:
# obtener datos de CA de España. Salen los códigos de las estaciones
# de Calidad de aire (941)
spain.sites <- get_saq_sites() %>%
    filter(country == "spain",
           site %in% sites.100mil$"Código estación",
           site_type == "traffic",
           site_area == "urban",
           date_start <= start_dt,
           date_end >= end_dt,
           ) %>%
    select(site, site_name, latitude, longitude, elevation, 
           country, site_type, site_area, date_start, date_end)

In [11]:
removed.sites <- nrow(sites.100mil) - nrow(spain.sites)

print(paste("Se han eliminado", removed.sites, "estaciones"))
print(paste("Quedan", nrow(spain.sites), "estaciones para el estudio"))

[1] "Se han eliminado 26 estaciones"
[1] "Quedan 53 estaciones para el estudio"


Agrupamos en un solo data.frame toda la informacion relevante de las estaciones de estudio

In [16]:
sites.info <- get_saq_processes() %>%
    filter(site %in% spain.sites$site,
           variable %in% pollutants,
           date_start <= start_dt,
           #date_end >= end_dt,
          ) %>%
    select(process, site, variable, variable_long, 
           period, unit, observation_count
          )

sites.geo <- sites.100mil %>%
    select("Municipio", "Población", 
           "Estación tráfico", "Código estación") %>%
    rename(site = "Código estación",)

sites <- merge(x = sites.info, y = sites.geo, by = "site", all.x = TRUE)
sites <- merge(x = sites, y = spain.sites, by = "site", all.x = TRUE)[1:2, ]

In [17]:
head(sites)

,site,process,variable,variable_long,period,unit,observation_count,Municipio,Población,Estación tráfico,site_name,latitude,longitude,elevation,country,site_type,site_area,date_start,date_end
,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dttm>,<dttm>
1,es0041a,31140,no2,Nitrogen dioxide (air),day,ug.m-3,151,Bilbao,346843,María Diaz de Haro,DIRECCIÓN DE SALUD,43.25883,-2.94565,32,spain,traffic,urban,1986-04-01,2021-01-15 23:00:00
2,es0110a,31281,o3,Ozone (air),day,ug.m-3,4092,Bilbao,24350,Erandio,ERANDIO,43.30268,-2.97724,4,spain,traffic,urban,1997-01-01,2021-01-15 23:00:00


## Guardar Datos en xlsx <a id="save"></a>

In [9]:
write.csv(sites, "../data/csv/sitesAQ.csv", row.names=FALSE)